*In this file, we seek to automate parametrisation, running and results extraction of the osemosys project*

In [5]:
import subprocess
import os
from pathlib import Path
import gdxpds
import pandas as pd
import matplotlib.pyplot as plt
import re
from gdxpds import to_dataframes
import numpy as np
import seaborn as sns

In [ ]:
results_folder = "Results" #path to results folder

def extract_results(selected_file):
    """
    Loads corresponding gdx file into a an ordered dict of pandas dataframes.
    """
    print("Loading results...")
    results = []
    results_dir = os.path.join(os.getcwd(), results_folder)
    os.makedirs(results_dir, exist_ok=True)
    pattern = re.compile(r"results_SCEN(?P<scen>\w+)_DATA(?P<data>\w+)_COST(?P<cost>\w+)_STOR(?P<stor>yes|no)\.gdx")

    file_lookup = {}
    for filename in os.listdir(results_dir):
        match = pattern.match(filename)
        if match:

            key = (match.group("scen"), match.group("data"), match.group("cost"), match.group("stor"))
            file_lookup[key] = os.path.join(results_dir, filename)
    
    # Check if the selected file exists in the lookup
    key = tuple(selected_file)
    if key in file_lookup:
        filename = file_lookup[key]
        print(filename)
        results = to_dataframes(filename)
    else:
        print(f"{key} not found")
        results = None

    return results 

In [22]:
sns.set_theme(style="whitegrid", font_scale=1.3)
from matplotlib.ticker import MaxNLocator
from collections import OrderedDict

#functions, utilities
def plot_name(df, target, y_col="Level", x_col=None, title=None):
    if x_col is None:
        x_col = "YEAR"
    xlabel = x_col
    ylabel = y_col

    if title=="StorageLevelSeasonStart": #treat seasonal plots differently
        df["Year_Season"] = df["YEAR"].astype(str) + "_S" + df["SEASON"].astype(str)
        x_col = "Year_Season"
        x_label = x_col
        df.drop(columns=["SEASON", "YEAR"], inplace=True)


    fig, ax = plt.subplots(figsize=(16, 10))
    if target != None:
        all_targets = sorted(df[target].unique())
        palette = sns.color_palette("tab20", n_colors=len(df[target].unique()))
        colour_map = dict(zip(all_targets, palette))

        df = df.pivot_table(index=x_col, columns=target, values='Level', aggfunc='sum', fill_value=0)
        for col in df.columns:
            if col[:3] == "VIR" or col[:3] == "IMP": #remove technologies that we are not interested in
                continue
            if (df[col] != 0).any():
                ax.plot(df.index, df[col], label=col, color=colour_map[col], linewidth=2)
    else:
        ax.plot(df["YEAR"], df[y_col], label=title, linewidth=2)
        
    ax.set_xlabel(xlabel if xlabel else x_col)
    ax.set_ylabel(ylabel if ylabel else y_col)
    ax.grid(True)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))  # Reduce overcrowding on x-axis
    ax.tick_params(axis='x', rotation=45)
    ax.set_title(f"{title if title else None}", fontsize=14)
    plt.legend()
    fig.tight_layout()
    plt.show()


def plot_allusefulcharts(gdx_f, filter=None):
    x_col = "YEAR"
    y_col = "Level"

    TechnologyPlots = ["ProductionByTechnologyAnnual", "ProductionByTechnology", "CapitalInvestment", 
    "UseByTechnologyAnnual","OperatingCost", "RateOfProductionByTechnology", 
    "RateOfTotalActivity", "AnnualTechnologyEmission", "TotalCapacityAnnual"]
    FuelPlots = ["RateOfUse", "Trade", "UseAnnual"]
    StoragePlots = ["NewStorageCapacity", "CapitalInvestmentStorage", "StorageLevelYearStart", "StorageLevelYearFinish", "StorageUpperLimit"]
    EmissionPlots = ["AnnualEmissions"]
    singleplots = ["TotalDiscountedCost"]
    seasonplots = ["StorageLevelSeasonStart"]
    
    for t in TechnologyPlots:
        if filter!= None and t not in filter:
            continue
        df = gdx_f[t]
        df = df[df["Level"] > 0]
        plot_name(df, "TECHNOLOGY", y_col=y_col, x_col=x_col, title=t)

    for f in FuelPlots:
        if filter != None and f not in filter:
            continue
        df = gdx_f[f]
        df = df[df["Level"] > 0]
        plot_name(df, "FUEL", y_col=y_col, x_col=x_col, title=f)

    for s in StoragePlots:
        if filter != None and s not in filter:
            continue
        df = gdx_f[s]
        df = df[df["Level"] > 0]
        plot_name(df, "STORAGE", y_col=y_col, x_col=x_col, title=s)

    for e in EmissionPlots:
        if filter != None and e not in filter:
            continue
        df = gdx_f[e]
        df = df[df["Level"] > 0]
        plot_name(df, "EMISSION", y_col=y_col, x_col=x_col, title=e)

    for single in singleplots:
        if filter != None and single not in filter:
            continue
        df = gdx_f[single]
        df = df[df["Level"] > 0]
        plot_name(df, None, y_col=y_col, x_col=x_col, title=single)

    for seasonplot in seasonplots:
        if filter != None and seasonplot not in filter:
            continue
        df = gdx_f[seasonplot]
        df = df[df["Level"] > 0]
        plot_name(df, "STORAGE", y_col=y_col, x_col=x_col, title=seasonplot)



def run_extr_gams(scenario, data, cost, storage, value=0.0, solvermode="mip", message=None):
    subprocess.run([
        "gams",
        "osemosys.gms",
        f"--scen={scenario}",
        f"--data={data}",
        f"--storage={storage}",
        f"--cost={cost}",
        f"--value={value}",
        f"--solvermode={solvermode}"], check=True)
    
    res = extract_results(scenario, data, cost, storage)
    return res

In [ ]:
# running in gams is optional - tailored for current scenario exploration, rerun the block after running gams
selected_file = ["emicap0", "template", "cheapbees", "yes"] #selected file to load
orderedDict = extract_results(selected_file)
# Plot all useful charts
listmichele = [
    "AnnualEmissions", "AnnualTechnologyEmission", "CapitalInvestment",
    "CapitalInvestmentStorage", "OperatingCost", "ProductionByTechnologyAnnual",
    "StorageLevelYearStart", "StorageLevelYearFinish", "StorageUpperLimit",
    "TotalCapacityAnnual", "UseByTechnologyAnnual", "TotalDiscountedCost",
    "StorageLevelSeasonStart"
]

plot_allusefulcharts(orderedDict, filter=listmichele)


Loading results...
Results directory: /Users/hugosmart/Documents/Github/Hy-run/Results
/Users/hugosmart/Documents/Github/Hy-run results_SCENemicap0_DATAtemplate_COSTcheapbees_STORyes.gdx
/Users/hugosmart/Documents/Github/Hy-run/Results/results_SCENemicap0_DATAtemplate_COSTcheapbees_STORyes.gdx


In [ ]:
# Example usage:            # We can run gams from here too.
ordereddict = run_extr_gams(*selected_file)
df_toplot = ordereddict["TotalTechnologyAnnualActivity"]
plot_name(df_toplot, target="TECHNOLOGY", y_col="Level", x_col="YEAR", title="Total Technology Annual Activity")


# Plot all useful charts
listmichele = [
    "AnnualEmissions", "AnnualTechnologyEmission", "CapitalInvestment",
    "CapitalInvestmentStorage", "OperatingCost", "ProductionByTechnologyAnnual",
    "StorageLevelYearStart", "StorageLevelYearFinish", "StorageUpperLimit",
    "TotalCapacityAnnual", "UseByTechnologyAnnual", "TotalDiscountedCost",
    "StorageLevelSeasonStart"
]

#or plot all useful charts, helping me populate the "useful charts"
plot_allusefulcharts(ordereddict, filter=listmichele)

--- Job osemosys.gms Start 05/20/25 01:45:59 49.2.0 9b2b0844 DAX-DAC arm 64bit/macOS
--- Applying:
    /Library/Frameworks/GAMS.framework/Versions/49/Resources/gmsprmun.txt
--- GAMS Parameters defined
    Input /Users/hugosmart/Documents/Github/Hy-run/osemosys.gms
    ScrDir /Users/hugosmart/Documents/Github/Hy-run/225e/
    SysDir /Library/Frameworks/GAMS.framework/Versions/49/Resources/
    --scen base
    --data template
    --storage yes
    --cost bees
    --value 0.0
    --solvermode mip
Licensee: Massimo Tavoni                                 G250226+0003Ac-GEN
          Fondazione CMCC                                           DCE4109
          225e/gamslice.dat
          node:56901401 mem:8 cores:8 ipport:63830 v:2                     
          Course license for use within the course and related course work
          The expiration date of time-limited license is Sep 24, 2025
System information: 8 physical cores and 8 Gb memory detected
GAMS 49.2.0   Copyright (C) 1987-2025 

TypeError: 'NoneType' object is not subscriptable